In [108]:
# dependencies
using Polynomials

# First Implementation 
Direction: 
- Evaluate the function at various points around the edges of the rectangle. 
- Count how many cycles the argument completes while traversing around the rectangle.

Spot rectangle, and how many zeros are inside.
By tracking the winding number, we can locate the number of zeros in the holomorphic function

Wind(arg) = 1/(2*pi*im) * integrate(1/(x+im*y))

In [126]:
# define data structure of rectangle
struct Rect 
    btmlft::Any
    uprght::Any
end

struct Step
    height::Float32
    width::Float32
    step::Float32
end

In [131]:
# calculate height and width
function parseInput(input) 
    height = input.uprght[2] - input.btmlft[2]
    width = input.uprght[1] - input.btmlft[1]
    n = 20 # change n accordingly so argument analysis is comprehensive
    step = height/n
    info = Step(height, width, step)
end

parseInput (generic function with 1 method)

In [128]:
# evaluate the argument at various points starting at bottomleft and traversing one round
# store arguments in array

function argBox(f, input, info)
    # bottom left to top left
    i = input.btmlft[2]
    argArray = []
    while i < input.uprght[2] 
        arg = angle(f(input.btmlft[1]+im*i))  # this is where the input function comes in for evaluation. Check multipoint.ipynb
        complexpoint = input.btmlft[1]+im*i
        println("f($complexpoint) = $arg")
        push!(argArray,arg)
        i += info.step
    end
    
    # top left to top right
    j = input.btmlft[1]
    while j < input.uprght[1]
        arg = angle(f(j+im*input.uprght[2]))
        complexpoint = j+im*input.uprght[2]
        println("f($complexpoint) = $arg")
        push!(argArray,arg)
        j += info.step
    end
    
    # top right to bottom right
    i = input.uprght[2]
    while i > input.btmlft[2]
        arg = angle(f(input.uprght[1]+im*i))
        complexpoint = input.uprght[1]+im*i
        println("f($complexpoint) = $arg")
        push!(argArray,arg)
        i -= info.step
    end
    
    # bottom right to bottom left
    j = input.uprght[1]
    while j > input.btmlft[1]
        arg = angle(f(j+im*input.btmlft[2]))
        complexpoint = j+im*input.btmlft[2]
        println("f($complexpoint) = $arg")
        push!(argArray,arg)
        j -= info.step
    end
    
    # add the starting point to make the loop closed
    arg = angle(f(input.btmlft[1]+im*input.btmlft[2]))
    push!(argArray,arg)
    complexpoint = input.btmlft[1]+im*input.btmlft[2]
    println("f($complexpoint) = $arg")
    return argArray
end


argBox (generic function with 1 method)

In [129]:
# spot jumps (~>= 2pi or 6)
# the output of argBox(input) is an array of the arguments
# use a for loop to check if any of the pairs match the criteria
# increase count and return the final count. This is the number of zeros in the locus


function countJump(f, arr, info)
    len = length(arr)
    count = 0
    add = []
    dec = []
    
    for i in 1:(len - 1)
        if arr[i] - arr[i+1] > 4 # slightly less than 2pi
            count += 1
            push!(add, (arr[i],arr[i+1],i))
        elseif arr[i] - arr[i+1] < -4 # decrement if it goes clockwise (less than -6)
            count -= 1
            push!(dec, (arr[i],arr[i+1],i))
        end
    end
    return count, add, dec
end

countJump (generic function with 1 method)

In [23]:
# subdivide rectangle into four
# run recursion of countJump for sub-rectangles that have zeros
# repeat recursion until all of the jumps are found. 
# return the bottom left coordinates of the squares
    
 
function divideRec(f, input, info, res, count)
    height = info.height
    width = info.width
    
    # bottom left square
    x_bl = input.btmlft[1]
    y_bl = input.btmlft[2]
    x_tr = width/2 + info.btmlft[1]
    y_tr = height/2 + info.btmlft[2]
    new_input = ((x_bl,y_bl),(x_tr, y_tr))
    new_info = parseInput(new_input)
    new_arr = argBox(f, new_input, new_info)

    if countJump(f, new_arr, new_info) == 0 && count != 0 # add count to know when to push coordinate to result array
        push!(res, (new_info.btmlft[1],new_info.btmlft[2]))
        res
    elseif countJump(f, new_arr, new_info) > 0 # change to != 0 after debugging
        divideRec(f, new_input, new_info, (x_bl, y_bl), count+1) # res: the bottom left coordinate?
    end

    # top left square
    x_bl = input.btmlft[1]
    y_bl = input.btmlft[2]+height/2
    x_tr = width/2 + info.btmlft[1]
    y_tr = info.uprght[2]
    new_input = ((x_bl, y_bl),(x_tr, y_tr))
    new_info = parseInput(new_input)
    new_arr = argBox(f, new_input, new_info)
    
    if countJump(f, new_arr, new_info) == 0 && count != 0
        push!(res, (new_info.btmlft[1],new_info.btmlft[2]))
        res
    elseif countJump(f, new_arr, new_info) > 0 # change to != 0 after debugging
        divideRec(f, new_input, new_info, (x_bl, y_bl), count+1) 
    end
    
    # top right square
    x_mid = width/2 + info.btmlft[1]
    y_mid = height/2 + info.btmlft[2]
    new_input = ((x_mid, y_mid),(input.uprght[1], input.uprght[2]))
    new_info = parseInput(new_input)
    new_arr = argBox(f, new_input, new_info)
    
    if countJump(f, new_arr, new_info) == 0 && count != 0
        push!(res, (new_info.btmlft[1],new_info.btmlft[2]))
        res
    elseif countJump(f, new_arr, new_info) > 0 # change to != 0 after debugging
        divideRec(f, new_input, new_info, (x_bl, y_bl)) 
    end
    
    # bottom right square
    x_bl = width/2 + info.btmlft[1]
    y_bl = info.btmlft[2]
    x_tr = info.uprght[1]
    y_tr = height/2 + info.btmlft[2]
    new_input = ((x_bl, y_bl),(x_tr, y_tr))
    new_info = parseInput(new_input)
    new_arr = argBox(f, new_input, new_info)
    
    if countJump(f, new_arr, new_info) == 0 && count != 0
        push!(res, (new_info.btmlft[1],new_info.btmlft[2]))
        res
    elseif countJump(f, new_arr, new_info) > 0 # change to != 0 after debugging
        divideRec(f, new_input, new_info, (x_bl, y_bl))
    end

end



divideRec (generic function with 2 methods)

## Unit Tests
Test the algorithm properly. Polynomials are easy to put the zeros exactly where you want them. Try multiplying another function which has no zero in the region to make it more computationally expensive.

In [134]:
# first unit test with 4 zeros
t1 = fromroots([-1+2im,2+4im,-3im, 4+6im])
input = Rect((1,1), (10,10))
info = parseInput(input) # handles steps
arr = argBox(t1, input, info)
countJump(t1, arr, info) # == 2 #(1,4), (4,6)

f(1 + 1im) = 3.141592653589793
f(1.0f0 + 1.45f0im) = -3.01685
f(1.0f0 + 1.9000001f0im) = -2.8981817
f(1.0f0 + 2.3500001f0im) = -2.8151615
f(1.0f0 + 2.8000002f0im) = -2.808915
f(1.0f0 + 3.2500002f0im) = -2.9269967
f(1.0f0 + 3.7000003f0im) = 3.0726695
f(1.0f0 + 4.15f0im) = 2.6570594
f(1.0f0 + 4.6f0im) = 2.2512777
f(1.0f0 + 5.0499997f0im) = 1.9344976
f(1.0f0 + 5.4999995f0im) = 1.6876928
f(1.0f0 + 5.9499993f0im) = 1.4813452
f(1.0f0 + 6.399999f0im) = 1.3004248
f(1.0f0 + 6.849999f0im) = 1.139854
f(1.0f0 + 7.2999988f0im) = 0.9985029
f(1.0f0 + 7.7499986f0im) = 0.8758315
f(1.0f0 + 8.199999f0im) = 0.7706991
f(1.0f0 + 8.649999f0im) = 0.68132055
f(1.0f0 + 9.099998f0im) = 0.60559595
f(1.0f0 + 9.549998f0im) = 0.54142684
f(1.0f0 + 9.999998f0im) = 0.4868994
f(1 + 10im) = 0.48689923181126904
f(1.45f0 + 10.0f0im) = 0.25068864
f(1.9000001f0 + 10.0f0im) = 0.0072187055
f(2.3500001f0 + 10.0f0im) = -0.24243471
f(2.8000002f0 + 10.0f0im) = -0.49668694
f(3.2500002f0 + 10.0f0im) = -0.75336015
f(3.7000003f0 + 10.

(-2, Any[(3.14159, -3.01685, 1)], Any[(-2.927, 3.07267, 6), (-3.02936, 3.12032, 38), (-3.14159, 3.14159, 84)])

In [135]:
t2 = fromroots([1+2im,1+4im,3im, 4+6im]) * Polynomial([1, 2, 3], :x)
input2 = Rect((-10,0), (0,10))
info2 = parseInput(input2)
countJump(t2, input2, info2) # == 1

f(-10 + 0im) = 1.2249730841393358
f(-10.0f0 + 0.5f0im) = 0.95978075
f(-10.0f0 + 1.0f0im) = 0.6916488
f(-10.0f0 + 1.5f0im) = 0.42160097
f(-10.0f0 + 2.0f0im) = 0.15068889
f(-10.0f0 + 2.5f0im) = -0.120031245
f(-10.0f0 + 3.0f0im) = -0.3895212
f(-10.0f0 + 3.5f0im) = -0.65678203
f(-10.0f0 + 4.0f0im) = -0.9208727
f(-10.0f0 + 4.5f0im) = -1.1809262
f(-10.0f0 + 5.0f0im) = -1.4361613
f(-10.0f0 + 5.5f0im) = -1.6858919
f(-10.0f0 + 6.0f0im) = -1.9295315
f(-10.0f0 + 6.5f0im) = -2.1665952
f(-10.0f0 + 7.0f0im) = -2.3966978
f(-10.0f0 + 7.5f0im) = -2.6195495
f(-10.0f0 + 8.0f0im) = -2.83495
f(-10.0f0 + 8.5f0im) = -3.042781
f(-10.0f0 + 9.0f0im) = 3.040188
f(-10.0f0 + 9.5f0im) = 2.847569
f(-10 + 10im) = 2.6624734387219053
f(-9.5f0 + 10.0f0im) = 2.5333853
f(-9.0f0 + 10.0f0im) = 2.396328
f(-8.5f0 + 10.0f0im) = 2.2507055
f(-8.0f0 + 10.0f0im) = 2.0958848
f(-7.5f0 + 10.0f0im) = 1.9311991
f(-7.0f0 + 10.0f0im) = 1.7559503
f(-6.5f0 + 10.0f0im) = 1.5694178
f(-6.0f0 + 10.0f0im) = 1.370866
f(-5.5f0 + 10.0f0im) = 1.159

(-2, Any[], Any[(-3.04278, 3.04019, 18), (-2.9997, 2.99259, 65)])